# fft score cam visualization tool


### Imports

In [1]:
from dash import Dash, dcc, html, Input, Output, callback_context
import plotly.express as px
from PIL import Image
import numpy as np
import pandas as pd
import base64
import io

app = Dash(__name__)

app.layout = html.Div([
    html.H1("Upload Image for Heatmap"),
    dcc.Upload(
        id='upload-image',
        children=html.Button('Upload Image'),
        multiple=False
    ),
    dcc.Graph(id='image-heatmap', style={'height': '80vh', 'width': '100%'}),
    html.Button('Reset Zoom', id='btn-reset-zoom')
])

def parse_contents(contents):
    content_type, content_string = contents.split(',')
    decoded = base64.b64decode(content_string)
    img = Image.open(io.BytesIO(decoded))
    gray_img = img.convert('L')
    img_array = np.array(gray_img)
    df = pd.DataFrame(img_array)
    fig = px.imshow(df, color_continuous_scale='viridis')
    # Allow free zooming by disabling axis constraints
    fig.update_layout(
        xaxis=dict(constrain='range', autorange=True),
        yaxis=dict(constrain='range', autorange=True),
        margin={'t': 0, 'b': 0, 'l': 0, 'r': 0},
        autosize=True
    )
    return fig

@app.callback(
    Output('image-heatmap', 'figure'),
    Input('upload-image', 'contents')
)
def update_output(contents):
    if contents is not None:
        return parse_contents(contents)
    else:
        # Return an empty figure or some default figure
        return px.imshow(pd.DataFrame(), color_continuous_scale='viridis')

@app.callback(
    Output('image-heatmap', 'style'),
    [Input('image-heatmap', 'relayoutData'),
     Input('btn-reset-zoom', 'n_clicks')],
    prevent_initial_call=True
)
def resize_plot_on_zoom(relayout_data, n_clicks):
    if not callback_context.triggered:
        return dash.no_update
    
    prop_id = callback_context.triggered[0]['prop_id']
    if 'btn-reset-zoom' in prop_id:
        return {'height': '80vh', 'width': '100%'}
    elif 'image-heatmap.relayoutData' in prop_id:
        # Check if zooming in
        if relayout_data and ('xaxis.range[0]' in relayout_data or 'yaxis.range[0]' in relayout_data):
            return {'height': '90vh', 'width': '100%'}
    return {'height': '80vh', 'width': '100%'}

if __name__ == '__main__':
    app.run_server(debug=True)
